In [ ]:
#Importing the required libraries
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Subset
from torchvision import datasets, transforms, models
import numpy as np

In [ ]:
#configurations
IMG_SIZE = 224
NUM_CLASSES = 10
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 1
DATA_SAMPLE_SIZE = 5000
TEST_SAMPLE_SIZE = 1000

In [ ]:
#setting device to CPU
device = 'cpu'

Data Loading & Preprocessing

In [ ]:
def load_cifar10_datasets():
  """
    Loads CIFAR-10, applies transformations, and subsamples the data.
  """
  #defining the necessary transformations for Resnet-50
  transform = transforms.Compose([
      transforms.Resize(IMG_SIZE),
      transforms.ToTensor(),
      transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) #standard normalization for ImageNet trained models
  ])

  #load datasets
  train_dataset_full = datasets.CIFAR10(root='/data', train=True, download=True, transform=transform)
  test_dataset_full = datasets.CIFAR10(root='/data', train=False, download=True, transform=transform)

  #subsampling training data
  train_indices = np.arange(DATA_SAMPLE_SIZE)
  train_dataset = Subset(train_dataset_full, train_indices)

  #subsampling testing data
  test_indices = np.arange(TEST_SAMPLE_SIZE)
  test_dataset = Subset(test_dataset_full, test_indices)

  #create dataloaders
  train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2)
  test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2)

  return train_dataloader, test_dataloader

Model definition & freezing

In [ ]:
def build_transfer_model():
  """
    Loads a pre-trained ResNet-50, freezes the backbone, and replaces the head.
  """
  #loading the pre-trained ResNet-50 model
  model = models.resnet50(weights=models.ResNet50_Weights.IMAGENET1K_V1)

  #freezing all the parameters in the pre-trained model
  for param in model.parameters():
    param.requires_grad = False

  #replacing the fully connected layer (classification head)
  num_ftrs = model.fc.in_features #the original resnet50 models 50 features
  #creating a new full connected layer
  model.fc = nn.Linear(num_ftrs, NUM_CLASSES)

  #moving the model to the device
  model = model.to(device)

  return model

Training function

In [ ]:
def train_model(model, train_loader, test_loader):
  """
    Trains the model for the configured number of epochs.
  """
  #only optimizing the parameters in the new classification head
  optimizer = optim.Adam(model.fc.parameters(), lr=LEARNING_RATE)
  criterion = nn.CrossEntropyLoss()

  for epoch in range(NUM_EPOCHS):
    model.train() #setting model to training mode
    running_loss = 0.0

    for i, (inputs, labels) in enumerate(train_loader):
      inputs, labels = inputs.to(device), labels.to(device)

      #zero the parameter gradients
      optimizer.zero_grad()

      #forward pass
      outputs = model(inputs)
      loss = criterion(outputs, labels)

      #backward pass & optimization
      loss.backward()
      optimizer.step()

      running_loss += loss.item() * inputs.size(0)

      #printing loss every 100 batches for monitoring
      if (i+1)%100 == 0:
        print(f"Batch {i+1}/{len(train_loader)}, Loss: {loss.item():.4f}")

    epoch_loss = running_loss/len(train_loader.dataset)
    print(f"Epoch {epoch+1}/{NUM_EPOCHS} Loss: {epoch_loss:.4f}")

  #after training evaluate the final performance
  evaluate_model(model, test_loader)

In [ ]:
def evaluate_model(model, test_loader):
  """
    Evaluates the model's performance on the test set.
  """
  model.eval() #setting model to evaluation mode
  correct = 0
  total = 0

  with torch.no_grad(): #disabling gradient calculation during evaluation
    for inputs, labels in test_loader:
      inputs, labels = inputs.to(device), labels.to(device)
      outputs = model(inputs)
      _, predicted = torch.max(outputs.data, 1)
      total += labels.size(0)
      correct += (predicted==labels).sum().item()

  accuracy = 100*correct/total
  loss = nn.CrossEntropyLoss()(outputs, labels).item()

  print(f"Test Loss: {loss:.4f}")
  print(f"Test Accuracy: {accuracy:.2f}%")

Execution

In [ ]:
#loading & preparing data
train_loader, test_loader = load_cifar10_datasets()

100%|██████████| 170M/170M [00:04<00:00, 40.5MB/s]


In [ ]:
#build the model
model = build_transfer_model()

Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth


100%|██████████| 97.8M/97.8M [00:00<00:00, 122MB/s]


In [ ]:
#training & evaluating the model
train_model(model, train_loader, test_loader)

Epoch 1/1 Loss: 1.3722
Test Loss: 0.7436
Test Accuracy: 72.90%


In [14]:
MODEL_SAVE_PATH = "resnet18_transfer_learned.pth"

In [15]:
torch.save(model, MODEL_SAVE_PATH)